# Лабораторная 2

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("data\creditcard.csv")

In [11]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [18]:
class DecisionTree():
    def __init__():
        pass
    
    def trainTree(X, Y):
        pass
    
    def predictTree(model, X):
        pass